### Recommendation Table

In [3]:
import pandas as pd
import numpy as np

In [22]:
final_similarity_matrix = pd.read_parquet('data/final_item_similarity_matrix_sample.parquet')

In [23]:
user_item_matrix = pd.read_parquet('data/user_item_matrix_sample.parquet')
book_df = pd.read_parquet('data/book_eng.parquet')

In [24]:
final_similarity_matrix.columns = final_similarity_matrix.columns.astype('str')
final_similarity_matrix.index = final_similarity_matrix.index.astype('str')

In [25]:
from modules.helper.PredictRating import PredictRating

pr = PredictRating()
filled_matrix = pr.fill_user_item_matrix_nonparallel(user_item_matrix, final_similarity_matrix)

In [26]:
filled_matrix

book_id,1012959,1080513,1375925,1646536,1690826,1802466,1843325,2115046,2167181,2238834,2833823,2993119,3172076,606215,776434,849880,920766,997117
ID,,,,,,,,,,,,,,,,,,
1045,2.846082,3.18644,2.642367,2.961553,2.82374,2.905027,2.978427,-1,3.038196,2.876195,3.186111,2.631188,2.985617,2.52775,3.202836,2.638352,2.889512,2.795312
10560,1.69011,1.553621,1.45949,1.809491,1.636256,1.610419,1.699868,1.733707,1.647502,1.506399,1.371888,1.75502,1.697155,1.496549,1.591274,1.807547,-1,1.817123
1071,-1,2.323461,2.478734,2.748658,2.848449,2.671449,2.541167,2.846082,2.544592,2.594729,2.477833,2.654309,2.693087,2.591096,2.586675,2.728367,2.81685,2.546796
10848,2.154469,2.126855,1.833989,2.283505,2.096156,2.281505,2.255231,2.388494,2.095426,1.982161,2.073109,2.37989,-1,1.901945,2.12487,2.104044,2.262874,2.060664
10857,1.63702,1.74302,1.453513,1.764826,1.814361,1.618333,1.768722,1.583011,1.604808,1.424155,1.478104,1.502302,1.578033,1.754734,1.814806,-1,1.807547,1.910132
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9547,1.707649,1.911864,1.58542,1.776932,1.694244,1.743016,1.787056,-1,1.822918,1.725717,1.911667,1.578713,1.79137,1.51665,1.921701,1.583011,1.733707,1.677187
9605,1.4867,1.783743,1.619551,1.64061,1.46236,1.614583,1.696737,1.911667,1.664472,1.488677,-1,1.38687,1.554832,1.538324,1.809646,1.478104,1.371888,1.304949
9627,1.526755,1.925227,1.835887,1.720752,1.499901,1.911113,2.050024,1.822918,-1,1.573245,1.664472,1.624241,1.571569,1.588815,1.899729,1.604808,1.647502,1.703054


In [27]:
filled_matrix.T.loc[:, '1045'].sort_values(ascending = False)[:10]

book_id
776434     3.202836
1080513     3.18644
2833823    3.186111
2167181    3.038196
3172076    2.985617
1843325    2.978427
1646536    2.961553
1802466    2.905027
920766     2.889512
2238834    2.876195
Name: 1045, dtype: object

In [28]:
user_id_list = []
book_metadata_title_list = []
book_metadata_description_list = []
book_metadata_id_list = []
for user_id in filled_matrix.index:
    book_metadata_id = filled_matrix.T[user_id].sort_values(ascending = False)[:10].index.tolist()
    book_metadata_id_list.extend(book_metadata_id)
    user_id_list.extend([user_id]*len(book_metadata_id))
    for book_id in book_metadata_id:
        book_metadata_title_list.append(book_df[book_df['Id']==int(book_id)]['Name'].values[0])
        book_metadata_description_list.append(book_df[book_df['Id']==int(book_id)]['Description'].values[0])

top_k_recommendations_df = pd.DataFrame({
    'user_id': user_id_list, 
    'book_id': book_metadata_id_list, 
    'book_title': book_metadata_title_list, 
    'description': book_metadata_description_list
    })

In [29]:
top_k_recommendations_df

,user_id,book_id,book_title,description
0,1045,776434,"Wild Cards (Wild Cards, #1)",A shared-universe superhero prose anthology ed...
1,1045,1080513,"Incubus Dreams (Anita Blake, Vampire Hunter, #12)",As consultant to the Regional Preternatural Cr...
2,1045,2833823,Talking to Dragons (Enchanted Forest Chronicle...,"One day, Daystar's mom, Cimorene, hands him a ..."
3,1045,2167181,"The Spellman Files (The Spellmans, #1)","Meet Isabel ""Izzy"" Spellman, private investiga..."
4,1045,3172076,The Two Noble Kinsmen,"Based on Chaucer's Knight's Tale, The Two Nobl..."
...,...,...,...,...
835,9805,3172076,The Two Noble Kinsmen,"Based on Chaucer's Knight's Tale, The Two Nobl..."
836,9805,1802466,"A Question of Proof (Nigel Strangeways, #1)",The faculty and student body at Sudeley are sh...
837,9805,2993119,The Reasons of Love,This beautifully written book by one of the wo...
838,9805,2238834,Love and Biology at the Center of the Universe,When she learns that her college sweetheart hu...
